In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import joblib
import sklearn  # Import the sklearn module to access its version
import xgboost  # Import the xgboost module to access its version

# Check versions
print("Pandas version:", pd.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("XGBoost version:", xgboost.__version__)
print("Joblib version:", joblib.__version__)

Pandas version: 1.5.3
Scikit-learn version: 1.2.2
XGBoost version: 1.7.6
Joblib version: 1.3.2


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/2. Audio Songs /1. Emotion Classification Model/Data Files/song_emotion.csv')

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,perceptr_mean,perceptr_var,tempo,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,mfcc5_mean,mfcc5_var,mfcc6_mean,mfcc6_var,mfcc7_mean,mfcc7_var,mfcc8_mean,mfcc8_var,mfcc9_mean,mfcc9_var,mfcc10_mean,mfcc10_var,mfcc11_mean,mfcc11_var,mfcc12_mean,mfcc12_var,mfcc13_mean,mfcc13_var,mfcc14_mean,mfcc14_var,mfcc15_mean,mfcc15_var,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,Emotion
0,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,9.015054e+05,0.083045,0.000767,-4.529724e-05,0.008172,0.000008,0.005698,123.046875,-113.570648,2564.207520,121.571793,295.913818,-19.168142,235.574432,42.366421,151.106873,-6.364664,167.934799,18.623499,89.180840,-13.704891,67.660492,15.343150,68.932579,-12.274110,82.204201,10.976572,63.386311,-8.326573,61.773094,8.803792,51.244125,-3.672300,41.217415,5.747995,40.554478,-5.162882,49.775421,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,Reflective Melodies
1,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,2.977893e+06,0.056040,0.001448,1.395807e-04,0.005099,-0.000178,0.003063,67.999589,-207.501694,7764.555176,123.991264,560.259949,8.955127,572.810913,35.877647,264.506104,2.907320,279.932922,21.510466,156.477097,-8.560436,200.849182,23.370686,142.555954,-10.099661,166.108521,11.900497,104.358612,-5.555639,105.173630,5.376327,96.197212,-2.231760,64.914291,4.220140,73.152534,-6.012148,52.422142,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,Energetic Vibes
2,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,7.840345e+05,0.076291,0.001007,2.105576e-06,0.016342,-0.000019,0.007458,161.499023,-90.722595,3319.044922,140.446304,508.765045,-29.093889,411.781219,31.684334,144.090317,-13.984504,155.493759,25.764742,74.548401,-13.664875,106.981827,11.639934,106.574875,-11.783643,65.447945,9.718760,67.908859,-13.133803,57.781425,5.791199,64.480209,-8.907628,60.385151,-1.077000,57.711136,-9.229274,36.580986,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,Reflective Melodies
3,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,1.493194e+06,0.033309,0.000423,4.583644e-07,0.019054,-0.000014,0.002712,63.024009,-199.544205,5507.517090,150.090897,456.505402,5.662678,257.161163,26.859079,158.267303,1.771399,268.034393,14.234031,126.794128,-4.832006,155.912079,9.286494,81.273743,-0.759186,92.114090,8.137607,71.314079,-3.200653,110.236687,6.079319,48.251999,-2.480174,56.799400,-1.079305,62.289902,-2.870789,51.651592,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,Energetic Vibes
4,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,1.572978e+06,0.101461,0.001954,-1.756129e-05,0.004814,-0.000010,0.003094,135.999178,-160.337708,5195.291992,126.219635,853.784729,-35.587811,333.792938,22.148071,193.456100,-32.478600,336.276825,10.852294,134.831573,-23.352329,93.257095,0.498434,124.672127,-11.793437,130.073349,1.207256,99.675575,-13.088418,80.254066,-2.813867,86.430626,-6.933385,89.555443,-7.552725,70.943336,-9.164666,75.793404,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,Energetic Vibes


In [ ]:
X = df.drop(['Emotion'], axis=1)
y = df['Emotion']

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
n_components = 10  # Use the same number of components as before
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

In [ ]:
grid_search = GridSearchCV(XGBClassifier(random_state=42), param_grid, cv=3, verbose=2)
grid_search.fit(X_train_pca, y_train)

# Get the best XGBoost model from hyperparameter tuning
best_xgb_model = grid_search.best_estimator_

# Evaluate the best XGBoost model
y_pred = best_xgb_model.predict(X_test_pca)
print("Classification Report for Best XGBoost Model:")
print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.7s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.6s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.6s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=4, n_es

In [ ]:
best_model_path = '/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/2. Audio Songs /1. Emotion Classification Model/Emotion_Classification_Model.pkl'
scaler_path = '/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/2. Audio Songs /1. Emotion Classification Model/Emotion_Scaler.pkl'
pca_path = '/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/2. Audio Songs /1. Emotion Classification Model/Emotion_PCA.pkl'
label_encoder_path = '/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/2. Audio Songs /1. Emotion Classification Model/Emotion_LabelEncoder.pkl'

# Save the best XGBoost model
joblib.dump(best_xgb_model, best_model_path)
print(f"Best XGBoost model saved as {best_model_path}")

# Save the fitted StandardScaler
joblib.dump(scaler, scaler_path)
print(f"Fitted Scaler saved as {scaler_path}")

# Save the fitted PCA model
joblib.dump(pca, pca_path)
print(f"Fitted PCA model saved as {pca_path}")

# Save the LabelEncoder
joblib.dump(label_encoder, label_encoder_path)
print(f"LabelEncoder saved as {label_encoder_path}")

Best XGBoost model saved as /content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/2. Audio Songs /1. Emotion Classification Model/Emotion_Classification_Model.pkl
Fitted Scaler saved as /content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/2. Audio Songs /1. Emotion Classification Model/Emotion_Scaler.pkl
Fitted PCA model saved as /content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/2. Audio Songs /1. Emotion Classification Model/Emotion_PCA.pkl
LabelEncoder saved as /content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/2. Audio Songs /1. Emotion Classification Model/Emotion_LabelEncoder.pkl
